# 新手任务——故事上下句生成

# 0.Intro

开个烂坑（雾
<br>
应该说是给acci_nlp的新手任务开个头，具体要求：
![note_1](pic/note_1.jpg)

因为用jupyter用的比较多，打算先在这里实现一下，再思考下怎么写成完整的项目

# 1.load data

In [1]:
import io
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [ ]:
o_data = 'origin_data/'
n_data = 'data/'
